In [1]:
from tabulate import tabulate
from sagas.crmsfa.odoo_facade import odoo, login
import sagas.crmsfa.odoo_info as info
from sagas.tool.app_settings import setup_logger, setup_jupyter

setup_logger('crmsfa.log')
setup_jupyter()

In [2]:
login()
info.desc_model("crm.lead")

+------------------------+-----------+-------------------------+
| name                   | type      | string                  |
|------------------------+-----------+-------------------------|
| city                   | char      | City                    |
| type                   | selection | Type                    |
| activity_state         | selection | Activity State          |
| order_ids              | one2many  | Orders                  |
| write_date             | datetime  | Last Updated on         |
| activity_date_deadli.. | date      | Next Activity Deadline  |
| mobile                 | char      | Mobile                  |
| partner_address_name   | char      | Partner Contact Name    |
| create_uid             | many2one  | Created by              |
| team_id                | many2one  | Sales Team              |
| email_cc               | text      | Global CC               |
| is_blacklisted         | boolean   | Blacklist               |
| activity_user_id       

In [5]:
model_name="crm.lead"

Model = odoo.env[model_name]
model_ids = Model.search([])
print(model_ids)
for record in Model.browse(model_ids):
    print(getattr(record, "name"), getattr(record, "email_from"))

[27, 26, 25, 21, 20, 16, 14, 10, 6, 5, 4, 3, 44, 43, 42, 41, 40, 39, 38, 37, 36, 35, 34, 33, 32, 31, 30, 29, 22, 19, 18, 17, 15, 13, 8, 2, 1, 24, 23]
Interest in your products info@agrolait.com
Open Space Design info@agrolait.com
Modern Open Space henry@elight.com
Office Design and Architecture False
Distributor Contract john.b@tech.info
Global Solutions: Furnitures False
Office Design Project info@agrolait.com
Design Software Info jga@solar.example.com
Opensides: Need Info tina@opensides.example.com
Quotation for 50 Chairs helle@stonageit.example.com
Campbell: Chairs hmc@yahoo.example.com
Pricing for 25 desks contact@thekompany.example.com
Quote for 600 Chairs tbrockhouse0@google.pl
Modernize old offices fseiller1@theglobeandmail.com
Furnitures for new location ngrabert2@dailymail.co.uk
Need a price: urgent ikirvell3@gnu.org
Custom Desks (100 pieces) credford4@salon.com
Cleaning subscription eespinazo5@reuters.com
Office chairs jjobbins6@simplemachines.org
Design New Shelves akalinovi

In [45]:
lead_ids = Model.search([('email_from', '=', 'vauxoo@yourcompany.example.com')], limit=1)
print(lead_ids)     
leads = Model.browse(lead_ids)
print(leads)
print(leads[0])
for lead in leads:
    print(lead.id, lead.name, type(lead.tag_ids))
    for tag in lead.tag_ids:
        print("\t", tag.name)

[19]
Recordset('crm.lead', [19])
Recordset('crm.lead', [19])
19 Interest in your customizable Pcs Model('crm.lead.tag')
	 产品


In [50]:
Model = odoo.env["calendar.event"]
model_ids = Model.search([])
print("total", len(model_ids))
for rec in Model.browse(model_ids):
    # <class 'datetime.datetime'>
    print(rec.id, rec.start, rec.stop, rec.duration)

total 7
7 2018-12-16 00:00:00 2018-12-16 00:00:00 8.5
6 2018-12-18 02:00:00 2018-12-18 10:30:00 8.5
5 2018-12-22 00:00:00 2018-12-22 00:00:00 0.0
4 2018-12-20 08:00:00 2018-12-20 10:30:00 2.5
3 2018-12-12 15:55:05 2018-12-12 18:55:05 3.0
2 2018-12-05 12:00:00 2018-12-05 19:00:00 7.0
1 2018-12-03 10:20:00 2018-12-03 16:30:00 6.3


In [67]:
import dateparser
import time
# procs-dateparser.md
print(dateparser.parse('1 hour ago'))
print(dateparser.parse(u'2小时前'))
print(dateparser.parse(u'2018年11月1日'))
print(dateparser.parse(u'2018年12月30日'))
print(time.strftime('%Y-%m-05 12:00:00'))

#⊕ [python - How to overcome "datetime.datetime not JSON serializable"? - Stack Overflow](https://stackoverflow.com/questions/11875770/how-to-overcome-datetime-datetime-not-json-serializable)
from datetime import date, datetime
from json import dumps

def json_serial(obj):
    """JSON serializer for objects not serializable by default json code"""

    if isinstance(obj, (datetime, date)):
        return obj.isoformat()
    raise TypeError ("Type %s not serializable" % type(obj))

print(dumps(datetime.now(), default=json_serial))

2019-01-14 19:57:34.214176
2019-01-14 18:57:34.216694
2018-11-01 00:00:00
2018-12-30 00:00:00
2019-01-05 12:00:00
"2019-01-14T20:57:34.224492"


In [68]:
a_date=str(dateparser.parse(u'2018年11月1日'))
b_date=str(dateparser.parse(u'2018年12月10日'))
# procs-odoo-orm-api.md
#⊕ [Get list of ids - Odoo - Stack Overflow](https://stackoverflow.com/questions/45839212/get-list-of-ids-odoo)
# ev_recs = Model.search([('start','<=',b_date),('start','>=',a_date)]).ids
ev_recs = Model.search([('start','<=',b_date),('start','>=',a_date)])
print(len(ev_recs))

2


In [31]:
import graphene
class CrmLead(graphene.ObjectType):
    name = graphene.String()
    email_from=graphene.String()
    
class Query(graphene.ObjectType):
    crm_lead = graphene.Field(CrmLead, email_from=graphene.String())
    crm_leads = graphene.List(CrmLead)
    
    def resolve_crm_lead(self, info, email_from):
        return get_crm_lead(email_from)
    def resolve_crm_leads(self, info):
        model_name="crm.lead"
        Model = odoo.env[model_name]
        model_ids = Model.search([])
        recordset=[]
        for record in Model.browse(model_ids):
            recordset.append(CrmLead(name=getattr(record, "name"), 
                                     email_from=getattr(record, "email_from")))
        return recordset
    
def get_crm_lead(sent):
    model_name="crm.lead"
    Model = odoo.env[model_name]
    lead_ids = Model.search([('email_from', '=', sent)], limit=1)   
    leads = Model.browse(lead_ids)
    assert len(leads)==1
    # print(leads)
    lead=leads[0]
    return CrmLead(name=lead.name, email_from=lead.email_from)

schema = graphene.Schema(query=Query)     

In [32]:
import json
default_query = '''
query FetchCrmLead($sent: String!) {
      crmLead(emailFrom: $sent) {
        name
        emailFrom
      }
    }
'''.strip()
variables = {"sent":"vauxoo@yourcompany.example.com"}
result = schema.execute(default_query, variables=variables)
print(json.dumps(result.data, indent=2, ensure_ascii=False))

{
  "crmLead": {
    "name": "Interest in your customizable Pcs",
    "emailFrom": "vauxoo@yourcompany.example.com"
  }
}


In [33]:
default_query = '''
{
  crmLeads {
    name
    emailFrom
  }
}
'''.strip()
result = schema.execute(default_query)
print(json.dumps(result.data, indent=2, ensure_ascii=False))

{
  "crmLeads": [
    {
      "name": "Interest in your products",
      "emailFrom": "info@agrolait.com"
    },
    {
      "name": "Would appreciate more information about your products",
      "emailFrom": "info@agrolait.com"
    },
    {
      "name": "Need to customize the solution",
      "emailFrom": "henry@elight.com"
    },
    {
      "name": "“Resource Planning” project develpment",
      "emailFrom": "false"
    },
    {
      "name": "Interest in your Partnership Contract",
      "emailFrom": "john.b@tech.info"
    },
    {
      "name": "Need a new design for my website",
      "emailFrom": "false"
    },
    {
      "name": "Interest in your Graphic Design Project",
      "emailFrom": "info@agrolait.com"
    },
    {
      "name": "Information about laptop",
      "emailFrom": "jga@solar.example.com"
    },
    {
      "name": "Need Info about your Services",
      "emailFrom": "tina@opensides.example.com"
    },
    {
      "name": "Need a Quotation for Computers with A

In [74]:
info.desc_model("calendar.event")

+------------------------+-----------+--------------------------+
| name                   | type      | string                   |
|------------------------+-----------+--------------------------|
| opportunity_id         | many2one  | Opportunity              |
| start_date             | date      | Start Date               |
| state                  | selection | Status                   |
| message_unread         | boolean   | Unread Messages          |
| start_datetime         | datetime  | Start DateTime           |
| write_date             | datetime  | Last Updated on          |
| th                     | boolean   | Thu                      |
| start                  | datetime  | Start                    |
| display_name           | char      | Display Name             |
| stop_datetime          | datetime  | End Datetime             |
| allday                 | boolean   | All Day                  |
| name                   | char      | Meeting Subject          |
| user_id 

In [97]:
import graphene
from odoorpc.fields import Many2many, Many2one, One2many, Unknown
TYPES_TO_FIELDS = {
    'binary': graphene.String,
    'boolean': graphene.Boolean,
    'char': graphene.String,
    'date': graphene.types.datetime.Date,
    'datetime': graphene.types.datetime.DateTime,
    'float': graphene.Float,
    'html': graphene.String,
    'integer': graphene.Int,
    
    #⊕ [Graphene-Python](http://docs.graphene-python.org/en/latest/types/scalars/#custom-scalars)
    'many2many': Many2many,
    'many2one': Many2one,
    'one2many': One2many,
    
    'reference': graphene.String,
    'selection': graphene.String,
    'text': graphene.String,
}

def generate_field(name, data):
    """Generate a well-typed field according to the data dictionary supplied
    (obtained via the `fields_get' method of any models).
    """
    assert 'type' in data
    field = TYPES_TO_FIELDS.get(data['type'], Unknown)()
    return field

In [99]:
field = TYPES_TO_FIELDS.get("date", Unknown)()
print(type(field))

Date


In [89]:
# from graphene.utils.str_converters import to_camel_case, to_snake_case

def to_camel_case(snake_str):
    components = snake_str.split("_")
    # We capitalize the first letter of each component except the first one
    # with the 'capitalize' method and join them together.
    return "".join(x.capitalize() if x else "_" for x in components[:])

def normalize_class(model):
    cls_name = model.replace('.', '_')
    return to_camel_case(cls_name)
print(normalize_class("calendar.event"))

CalendarEvent


In [100]:
from odoorpc.env import FIELDS_RESERVED
import odoorpc.fields as fields

model="calendar.event"
attrs = {
        '_name': model,
        '_columns': {},
    }
def get_recurrent_fields():
    return ['byday', 'recurrency', 'final_date', 'rrule_type', 'month_by',
            'interval', 'count', 'end_type', 'mo', 'tu', 'we', 'th', 'fr', 'sa',
            'su', 'day', 'week_list']
fields_get = odoo.execute(model, 'fields_get')
for field_name, field_data in fields_get.items():
    field_type=field_data['type']
    if field_name not in FIELDS_RESERVED and \
        field_name not in get_recurrent_fields() and \
        not field_name.startswith("_") and \
        field_type not in ("many2many","many2one","one2many"):
            
        Field = fields.generate_field(field_name, field_data)
        FieldType=generate_field(field_name, field_data)
        attrs['_columns'][field_name] = Field
        attrs[field_name] = FieldType
        print(field_name, type(FieldType))

cls_name=normalize_class(model)
print(".. create class", cls_name)
model_cls=type(cls_name, (graphene.ObjectType,), attrs)

start_date Date
state String
message_unread Boolean
start_datetime DateTime
write_date DateTime
start DateTime
display_name String
stop_datetime DateTime
allday Boolean
name String
stop_date Date
recurrent_id Int
res_id Int
rrule String
location String
stop DateTime
recurrent_id_date DateTime
attendee_status String
active Boolean
privacy String
duration Float
message_needaction Boolean
message_last_post DateTime
res_model String
is_highlighted Boolean
display_time String
display_start String
message_needaction_counter Int
show_as String
create_date DateTime
message_is_follower Boolean
message_unread_counter Int
is_attendee Boolean
description String
.. create class CalendarEvent


In [108]:
for fld in model_cls._columns.keys():
    print(fld)

start_date
state
message_unread
start_datetime
write_date
start
display_name
stop_datetime
allday
name
stop_date
recurrent_id
res_id
rrule
location
stop
recurrent_id_date
attendee_status
active
privacy
duration
message_needaction
message_last_post
res_model
is_highlighted
display_time
display_start
message_needaction_counter
show_as
create_date
message_is_follower
message_unread_counter
is_attendee
description


In [117]:
import inspect

def desc_type(type_name):
    # attributes = inspect.getmembers(type(type_name), lambda a:not(inspect.isroutine(a)))
    attributes = inspect.getmembers(type_name, lambda a:not(inspect.isroutine(a)))    
    fields=[a for a in attributes 
            if not(a[0].startswith('__') 
                   and a[0].endswith('__'))]
    for f in fields:
        if f[0]!="_columns":
            print(f)    

desc_type(model_cls)

('_meta', <ObjectTypeOptions name='CalendarEvent'>)
('_name', 'calendar.event')
('active', <graphene.types.scalars.Boolean object at 0x1065b2e80>)
('allday', <graphene.types.scalars.Boolean object at 0x1065b25f8>)
('attendee_status', <graphene.types.scalars.String object at 0x1065b2240>)
('create_date', <graphene.types.datetime.DateTime object at 0x1069f9c18>)
('description', <graphene.types.scalars.String object at 0x106509d68>)
('display_name', <graphene.types.scalars.String object at 0x10530f2b0>)
('display_start', <graphene.types.scalars.String object at 0x1069f9b00>)
('display_time', <graphene.types.scalars.String object at 0x1069f9588>)
('duration', <graphene.types.scalars.Float object at 0x1065b2710>)
('is_attendee', <graphene.types.scalars.Boolean object at 0x1069f90f0>)
('is_highlighted', <graphene.types.scalars.Boolean object at 0x1069f9c50>)
('is_type_of', None)
('location', <graphene.types.scalars.String object at 0x1065b2e48>)
('message_is_follower', <graphene.types.scalar

In [118]:
import graphene

def create_with_attrs(model_cls, record):
    inst=model_cls()
    for fld in model_cls._columns.keys():
        val=getattr(record, fld)
        setattr(inst, fld, val)
    return inst

model_name="calendar.event"
of_type=model_cls

class CalQuery(graphene.ObjectType):
    calendar_event = graphene.List(of_type)
    
    def resolve_calendar_event(self, info):        
        Model = odoo.env[model_name]
        model_ids = Model.search([])
        recordset=[]
        for record in Model.browse(model_ids):
            recordset.append(create_with_attrs(of_type, record))
        return recordset

schema = graphene.Schema(query=CalQuery)    
print(of_type)

CalendarEvent


In [120]:
default_query = '''
{
  calendarEvent {
    name
    description
    start
  }
}
'''.strip()
result = schema.execute(default_query)
print(json.dumps(result.data, indent=2, ensure_ascii=False))

{
  "calendarEvent": [
    {
      "name": "Presentation of the new Calendar",
      "description": "false",
      "start": "2018-12-16T00:00:00"
    },
    {
      "name": "Presentation for new Services",
      "description": "false",
      "start": "2018-12-18T02:00:00"
    },
    {
      "name": "Changes in Designing",
      "description": "false",
      "start": "2018-12-22T00:00:00"
    },
    {
      "name": "Requirements review",
      "description": "false",
      "start": "2018-12-20T08:00:00"
    },
    {
      "name": "Pricing Discussion",
      "description": "Internal meeting for discussion for new pricing for product and services.",
      "start": "2018-12-12T15:55:05"
    },
    {
      "name": "Initial discussion",
      "description": "Discussion with partner for product.",
      "start": "2018-12-05T12:00:00"
    },
    {
      "name": "Follow-up for Project proposal",
      "description": "Meeting to discuss project plan and hash out the details of implementation.",
